In [ ]:
%%capture

import pandas as pd
import altrair as alt
import new_transit_metrics
from calitp_data_analysis import calitp_color_palette as cp
from IPython.display import HTML, Markdown, display

# alt.renderers.enable("html")
alt.data_transformers.enable("default", max_rows=None)

WIDTH = 300
HEIGHT = 150

In [ ]:
rtpa= "Metropolitan Transportation Commission"

In [ ]:
%%capture_parameters
rtpa